In [1]:
!wget -O ngrok.zip https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.zip
!unzip ngrok.zip
!chmod +x ngrok.;*/
!pip install pyngrok
!pip install chess

--2025-06-22 09:34:37--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 99.83.220.108, 35.71.179.82, 13.248.244.96, ...
Connecting to bin.equinox.io (bin.equinox.io)|99.83.220.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9638576 (9.2M) [application/octet-stream]
Saving to: ‘ngrok.zip’

ngrok.zip           100%[===================>]   9.19M  11.0MB/s    in 0.8s    

2025-06-22 09:34:39 (11.0 MB/s) - ‘ngrok.zip’ saved [9638576/9638576]

Archive:  ngrok.zip
  inflating: ngrok                   
chmod: cannot access 'ngrok.': No such file or directory
/bin/bash: line 1: sample_data/: Is a directory
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 57.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147775 sha256=f2427b037450ea528a401d13b17e0dc0433f00bc2aa499e13f55ec68f229c257
  Stored in directory: /root

In [2]:
!./ngrok authtoken 2uFxz9fixy7Pf18kdVMF605w399_27M5Ysckn4fGHrdKd1RYp


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import os
import chess
import chess.engine
import requests
from flask import Flask, request, jsonify
from pyngrok import ngrok  # Install with `pip install pyngrok`

app = Flask(__name__)

# Piece values for evaluation
piece_values = {
    chess.PAWN: 100, chess.KNIGHT: 320, chess.BISHOP: 330,
    chess.ROOK: 500, chess.QUEEN: 900, chess.KING: 20000
}

def evaluate_board(board):
    score = 0
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            value = piece_values.get(piece.piece_type, 0)
            score += value if piece.color == chess.WHITE else -value
    return score

def alpha_beta_pruning(board, depth, alpha, beta, maximizing_player):
    if depth == 0 or board.is_game_over():
        return evaluate_board(board)

    legal_moves = list(board.legal_moves)
    if maximizing_player:
        max_eval = float('-inf')
        for move in legal_moves:
            board.push(move)
            eval = alpha_beta_pruning(board, depth - 1, alpha, beta, False)
            board.pop()
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break  # Beta cut-off
        return max_eval
    else:
        min_eval = float('inf')
        for move in legal_moves:
            board.push(move)
            eval = alpha_beta_pruning(board, depth - 1, alpha, beta, True)
            board.pop()
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break  # Alpha cut-off
        return min_eval

# @app.route("/best_move", methods=["POST"])
def find_best_move(fen, depth=5):
    board = chess.Board(fen)
    best_move = None
    best_value = float('-inf') if board.turn == chess.WHITE else float('inf')

    for move in board.legal_moves:
        board.push(move)
        board_value = alpha_beta_pruning(board, depth - 1, float('-inf'), float('inf'), not board.turn)
        board.pop()

        if (board.turn == chess.WHITE and board_value > best_value) or (board.turn == chess.BLACK and board_value < best_value):
            best_value = board_value
            best_move = move  # <-- This is now a chess.Move object

    return best_move  # <-- Return a Move object instead of a string


@app.route('/best_move', methods=['POST'])
def get_best_move():
    data = request.get_json()
    fen = data.get('fen')
    depth = data.get('depth', 5)

    if not fen:
        return jsonify({"error": "FEN string is required"}), 400

    best_move = find_best_move(fen, depth)
    # return jsonify({"best_move": best_move})
    return jsonify({"best_move": best_move.uci() if best_move else None})

def run_flask():
    app.run(port=5000, use_reloader=False)  # Run Flask on port 5000


# Start ngrok programmatically
try:
    public_url = ngrok.connect(5000).public_url
    print(f"ngrok tunnel created: {public_url}")
except Exception as e:
    print(f"Error starting ngrok: {e}")

run_flask()
# app.stop()

ngrok tunnel created: https://6261-34-125-84-96.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [22/Jun/2025 09:36:50] "POST /best_move HTTP/1.1" 200 -
